We want to specify a phantom and scan geometry, noise model and generate some noisy data of known noise level. This means user can change parameters to play with different setups and investigate influence of parameters such as noise level easily.

Notebook should illustrate basics of how to set up and work with CT problems with CUQIpy-CIL.

Should show case/overview what is available incl 
- Parallel and fan-beam geometries.
- Models and test problems.

Set up and sample a Bayesian problem.  Probably not Gibbs.

Could also illustrate some of CIL's tools, e.g. show_geometry, islicer

Include some suggestions for exploration.
 - experiment to avoid inverse crime?

Things to vary:
- noise type added, and which data distribution to use, effect of wrong data distribution
- phantom smooth or piecewise constant, effect of wrong prior
- full or limited angle

## Demo 00 simple

In [ ]:
# %%
import cuqi
import cuqipy_cil
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load a CT forward model and data from testproblem
A, y_data, info = cuqipy_cil.testproblem.ParallelBeam2DProblem.get_components(
    im_size=(128, 128),
    det_count=128,
    angles=np.linspace(0, np.pi, 180),
    phantom="shepp-logan"
)

In [ ]:
# Set up Bayesian model
x = cuqi.distribution.GaussianCov(np.zeros(A.domain_dim), 1) # x ~ N(0, 1)
y = cuqi.distribution.GaussianCov(A@x, 0.05**2)              # y ~ N(Ax, 0.05^2)

In [ ]:
# Set up Bayesian Problem
BP = cuqi.problem.BayesianProblem(y, x)
BP.set_data(y=y_data)

In [ ]:
# Sample from the posterior
samples = BP.sample_posterior(200)

In [ ]:
# Analyze the samples
info.exactSolution.plot(); plt.title("Exact solution"); plt.show()
y_data.plot(); plt.title("Data"); plt.show()
samples.plot_mean(); plt.title("Posterior mean"); plt.show()
samples.plot_std(); plt.title("Posterior standard deviation"); plt.show()

In [ ]:
samples.plot()

## Demo 01

In [ ]:
# %%
# Load cuqi and other packages
import cuqi
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Specifically load the CT library components
from cuqipy_cil.model import ParallelBeam2DModel, FanBeam2DModel
from cuqipy_cil.testproblem import ParallelBeam2DProblem

In [ ]:
#%% Define CT model conveniently with cuqi
model = ParallelBeam2DModel() #CT model with parallel-beam and default values
model= FanBeam2DModel() #CT model with fan-beam and default values

## Can also illustrate CIL tools:

In [ ]:
import cil

In [ ]:
from cil.utilities.display import show2D, show_geometry

In [ ]:
show_geometry(model.acquisition_geometry)

In [ ]:
# Extract parameters from model
N   = model.domain_geometry.fun_shape[0]
n   = model.domain_geometry.par_dim #N*N
p,q = model.range_geometry.fun_shape
m   = model.range_geometry.par_dim  #p*q

In [ ]:
# %% Phantom
# Get exact phantom
x_exact = cuqi.data.shepp_logan(size = N)

In [ ]:
x_exact

In [ ]:
# Phantom in cuqi array with geometry
x_exact = cuqi.samples.CUQIarray(x_exact, is_par=False, geometry=model.domain_geometry)

In [ ]:
x_exact.plot()

In [ ]:
# Plot phantom
plt.figure()
x_exact.plot()
plt.colorbar()
#%% Generate exact data and plot it
b_exact = model.forward(x_exact)
plt.figure()
b_exact.plot()
plt.colorbar()

In [ ]:
#%% Plot back projection
plt.figure()
model.adjoint(b_exact).plot()
plt.colorbar()

In [ ]:
#%% Define Gaussian prior and data distribution
prior      = cuqi.distribution.GaussianCov(np.zeros(n), 0.5, geometry=model.domain_geometry, name="x")
data_dist  = cuqi.distribution.GaussianCov(model, 0.1, geometry=model.range_geometry, name="y")

In [ ]:
#%% Generate noisy data using the data distribution from x_exact
data=data_dist(x_exact).sample()
plt.figure()
data.plot()
plt.colorbar()

In [ ]:
#%% Construct likelihood function
likelihood = data_dist.to_likelihood(data)

In [ ]:
#%% Posterior distribution
posterior = cuqi.distribution.Posterior(likelihood, prior)

In [ ]:
#%% Sample posterior
sampler = cuqi.sampler.Linear_RTO(posterior)
samples = sampler.sample(500,100)

In [ ]:
#%% Plot mean
plt.figure()
samples.plot_mean()
plt.colorbar()
#%% Plot std
plt.figure()
samples.plot_std()
plt.colorbar()

In [ ]:
#%% Plot posterior samples
plt.figure()
samples.plot()
plt.colorbar()

In [ ]:
#%% High level test problem
BP = ParallelBeam2DProblem(prior=prior, noise_std=0.01, phantom="grains")

cuqi.config.MAX_DIM_INV = 1000 # Change max dim to a lower number such that the problem will be sampled using LinearRTO
samples_BP = BP.sample_posterior(500)

plt.figure()
samples_BP.plot_mean()
plt.colorbar()

plt.figure()
samples_BP.plot_std()
plt.colorbar()

# %%

## Demo 2

In [ ]:
# %%
import numpy as np
from cuqi.distribution import Cauchy_diff
from cuqipy_cil.testproblem import ParallelBeam2DProblem

In [ ]:
# Computed Tomography
TP = ParallelBeam2DProblem(
    im_size=(256, 256),
    det_count=256,
    angles=np.linspace(0, np.pi, 180),
    phantom="shepp-logan",
)

In [ ]:
# Cauchy difference prior
TP.prior = Cauchy_diff(
    location=np.zeros(TP.model.domain_dim),
    scale=0.01,
    physical_dim=2,
)

In [ ]:
# Sample posterior with automatic sampler choice
samples = TP.sample_posterior(200)

In [ ]:
# Plot sample mean and ci
samples.plot_ci()

## Ideas for exploration:
1. Set up a few-projection case, for example 40 projections over 180 degrees. Explore different priors.
2. Set up a limited angle case, for example instead of full 180-degree data, use 135, 90 or even 45 degree data, and explore different the effect of different priors on the solution and its uncertainty.
3. Try with a smooth phantom instead of a piecewise constant.
4. Infer noise level and/or prior precision through Gibbs.

# FIPS dataset

In [ ]:
ls

In [ ]:
import scipy
import numpy as np
import cil
from cil.framework import AcquisitionData, AcquisitionGeometry
from cil.utilities.display import show2D, show_geometry
from cil.recon import FDK
from cil.plugins.astra.processors import FBP

In [ ]:
sinogram = scipy.io.loadmat('sinogram.mat')['sinogram'].T

In [ ]:
sinogram = sinogram[0:360, 0:2221]

In [ ]:
sinogram.shape

In [ ]:
num_angles = sinogram.shape[0]
num_dets = sinogram.shape[1]
source_center = 54
source_detector = 63
det_pixel_size = 12/2240

In [ ]:
angles = np.linspace(0,360,num_angles,endpoint=False)

In [ ]:
ag = AcquisitionGeometry.create_Cone2D(source_position=[0,-source_center], 
                                      detector_position=[0,source_detector-source_center])\
    .set_panel(num_pixels=num_dets, pixel_size=det_pixel_size)\
    .set_angles(angles=-angles, angle_unit='degree')

In [ ]:
data = AcquisitionData(sinogram, geometry=ag)

In [ ]:
show_geometry(ag)

In [ ]:
ig = data.geometry.get_ImageGeometry()

In [ ]:
print(ig)

In [ ]:
from cil.plugins.astra import ProjectionOperator
from cil.optimisation.algorithms import CGLS

In [ ]:
AP = ProjectionOperator(ig,ag,device='cpu')

In [ ]:
cgls = CGLS(operator=AP, data=data, tolerance=-1, max_iteration=1000)

In [ ]:
cgls.run(5)

In [ ]:
show2D(cgls.solution)

In [ ]:
AP.adjoint(data)

In [ ]:
data.shape

In [ ]:
ig

In [ ]:
print(ig)

In [ ]:
2221**2

In [ ]:
from cil.processors import Binner

In [ ]:
data_binned = Binner(roi={'angle':(0,360,1),'horizontal':(0,2221,10)})(data)

In [ ]:
data_binned.shape

In [ ]:
show2D(data_binned)

In [ ]:
ag_b = data_binned.geometry

In [ ]:
ig_b = ag_b.get_ImageGeometry()

In [ ]:
AP_b = ProjectionOperator(ig_b,ag_b,device='cpu')
cgls = CGLS(operator=AP_b, data=data_binned, tolerance=-1, max_iteration=1000)

In [ ]:
cgls.run(17)

In [ ]:
show2D(cgls.solution)